# Artificial data generation

In this notebook, we show how to generate the artificial data sets used to train and test our models. 

## Asia model

We load in the Asia example model from the [BNlearn](http://bnlearn.com/bnrepository/discrete-small.html#asia) repository. We remove the either node and adapt the conditional probability tables accordingly, so the joint probability distribution remains the same. 

In [1]:
from pgmpy.utils import get_example_model
from pgmpy.factors.discrete import TabularCPD

def get_asia_model(): 

    """
    create the ground-truth asia model based on http://bnlearn.com/bnrepository/discrete-small.html#asia
    remove the "either" node and adapt CPTs accordingly
    """

    asia = get_example_model('asia')

    asia.remove_node("either")
    asia.add_edge(u="tub",v="dysp")
    asia.add_edge(u="tub",v="xray")
    asia.add_edge(u="lung",v="dysp")
    asia.add_edge(u="lung",v="xray")

    cpd_xray = TabularCPD(variable='xray', variable_card=2, 
                          values=[[0.98, 0.98, 0.98, 0.05], # xray "yes": [L=y,T=y],[L=y,T=n],[L=n,T=y],[L=n,T=n]
                                  [0.02, 0.02, 0.02, 0.95]], # xray "no": [id.]
                          evidence=['lung', 'tub'],
                          evidence_card=[2, 2],
                          state_names={'xray': ['yes', 'no'],
                                       'lung': ['yes', 'no'],
                                       'tub': ['yes', 'no']})

    asia.add_cpds(cpd_xray)

    cpd_dysp = TabularCPD(variable='dysp', variable_card=2, 
                          values=[[0.9, 0.9, 0.9, 0.8, 0.7, 0.7, 0.7, 0.1], # xray "yes": [B=y,L=y,T=y],[B=y,L=y,T=n],[B=y,L=n,T=y],[B=y,L=n,T=n],[B=n,L=y,T=y],[B=n,L=y,T=n],[B=n,L=n,T=y],[B=n,L=n,T=n]
                                  [0.1, 0.1, 0.1, 0.2, 0.3, 0.3, 0.3, 0.9]], # xray "no": [id.]
                          evidence=['bronc', 'lung', 'tub'],
                          evidence_card=[2, 2, 2],
                          state_names={'dysp': ['yes', 'no'],
                                       'bronc': ['yes', 'no'],
                                       'lung': ['yes', 'no'],
                                       'tub': ['yes', 'no']})

    asia.add_cpds(cpd_dysp)
    
    return asia

In [2]:
GT_model = get_asia_model()

## Train set

We obtain 20000 samples from the ground-truth joint distribution as a full training set (from which we can subsample to get training sets of smaller sizes). The training set is sampled in chunks of 1000 samples, since our experiments showed that the BayesianModelSampling method in the pgmpy library could get unstable if we extract a large (e.g. 20000) number of samples at once. 

The resulting train set is written to a file, where the first line indicates the variable that is represented by each position in the sample vector, and the second line indicates the possible classes per variable. The observed class per variable in a sample is indicated by a one-hot encoding, with the positions corresponding to the variable and classes as given by the first two lines of the file. 

In [7]:
from pgmpy.sampling import BayesianModelSampling
import random
import pandas as pd

def sample_dataset(model, N_samples, c_size, seed, file_name):
    
    # extract samples in chunks of size c_size 
    random.seed(seed)
    seed = random.randint(1, 1000)
    inference = BayesianModelSampling(model)
    samples = inference.forward_sample(size=c_size, seed=seed, show_progress=False)
    N_chunks = N_samples//c_size
    for _ in range(N_chunks-1):
        seed = random.randint(1, 1000)
        samp = inference.forward_sample(size=c_size, seed=seed, show_progress=False)
        samples = pd.concat([samples, samp], ignore_index=True)
    
    # create dataframe with one-hot encoding of samples
    states = model.states
    for key in states:
        for value in states[key]:
            samples[key + " " + value] = samples[key] == value
    for key in states: 
        del samples[key]
    samples = samples.replace(to_replace={True: 1.0, False: 0.0})
    
    # write resulting samples to file
    with open(file_name, 'w') as outfile: 
        var_names = [col_name.split(" ")[0] for col_name in samples.columns]
        state_names = [col_name.split(" ")[1] for col_name in samples.columns]
        outfile.write(','.join(var_names)+'\n')
        outfile.write(','.join(state_names)+'\n')
        for index, row in samples.iterrows(): 
            r = [str(e) for e in row]
            outfile.write(','.join(list(r))+'\n')

In [8]:
sample_dataset(GT_model, 20000, 1000, 2022, "data/asia/train_samples.txt")

## Test set: all queries

We create a test set to compute the total MAE. This means we generate all possible evidence combinations with all possible values, and obtain the conditional probability for each query from the ground-truth model. 

We write the result to a file, with each line representing one query. Again, the first two lines of the file contain the ordering of the variables and their states. Evidence settings are represented with integer one-hot encoding (either 1 or 0 for each of the states for a particular variable), while expected target probabilities are represented by a float (the targets belonging to the same variable all sum to one). When receiving such a file, the *TestQueryDataset* class is able to extract the evidence/target mask, the input vector (values of the evidence variables, targets are nan) and the target probability vector (evidence variables are nan). 

In [17]:
from pgmpy.inference import VariableElimination
import itertools
import numpy as np

def generate_all_queries(model):
    
    all_vars = list(model.nodes) # list of vars in the model
    all_states = model.states # dictionary: key=var, value=list of state names
    
    one_hot = {var:{state:i for i,state in enumerate(all_states[var])} for var in all_states} # specifies the index within the one-hot vector
                                                                                              # which is equal to 1 when a state is selected (per var)
                                                                                              # key=var, value=dict of state:index
    
    infer = VariableElimination(model) # inference object
    n = len(all_vars)
    df = [] # dataset which will contain all samples
    
    for r in range(1, n): # loop over r = number of vars included in evidence state in this iteration
        comb = itertools.combinations(all_vars, r) # get all sets of r vars (= evidence sets)
        
        for c in comb: 
            states = []
            for var in c: 
                states.append(all_states[var])
            prod = itertools.product(*states) # get all possible assignments of states to the set of vars in c
            
            for p in prod:
                
                evidence = {}
                for i in range(len(p)): 
                    evidence[c[i]] = p[i] # create the evidence set
                res = {}
                for e in evidence: # add one-hot vector to sample, representing which state was selected per evidence var
                    r = np.zeros(len(all_states[e]), dtype=int) 
                    r[one_hot[e][evidence[e]]] = 1
                    res[e] = r
                    
                for var in all_vars: # query probability for all vars which are not included in evidence
                    if var not in evidence: 
                        answer = infer.query([var], evidence=evidence, show_progress=False)
                        res[var] = answer.values # add array of probs to sample
                df.append(res) # add sample to dataset
                
    res = {}
    
    for var in all_vars:
        res[var] = infer.query([var], show_progress=False).values # add no-evidence probabilities (marginalize out all other vars)
    df.append(res)
    
    return df

In [18]:
def write_test_file(model, dataset, path):
    with open(path, 'w') as outfile: 
        all_states = model.states # dictionary: key=var, value=list of state names
        mapping_vars = ','.join([','.join(len(all_states[var])*[var]) for var in all_states])
        mapping_labels = ','.join([','.join(all_states[var]) for var in all_states])
        outfile.write(mapping_vars+'\n')
        outfile.write(mapping_labels+'\n')
        for r in dataset:
            line = []
            for var in all_states:
                if r[var].dtype != int:
                    line += ["{:.5f}".format(e) for e in r[var]]
                else:
                    line += [str(e) for e in r[var]]
            outfile.write(','.join(line)+'\n')

In [19]:
query_dataset = generate_all_queries(GT_model)
write_test_file(GT_model, query_dataset, "data/asia_new/total_test_set.txt")

## Test set: sampled queries

We gather 1000 test samples from the ground-truth distribution. For each of these, we generate a random evidence/target mask. The selected evidence variables are set to the observed value in the sample, while the conditional target probabilities are obtained from the ground-truth model for the remaining variables. We write the results to another file, with the same structure as the *total_test_set* file. 

In [21]:
Nsamples = 1000
inference = BayesianModelSampling(GT_model)
test_samples = inference.forward_sample(size=Nsamples, seed=2022) # use a different seed than sampling training set!

  0%|          | 0/7 [00:00<?, ?it/s]

In [22]:
def pred_query(model, evidence, target):
    
    all_states = model.states
    one_hot = {var:{state:i for i,state in enumerate(all_states[var])} for var in all_states}
    
    res = {}
    
    # add one-hot vector to sample representation, storing which state was selected per evidence var
    for e in evidence: 
        r = np.zeros(len(all_states[e]), dtype=int) 
        r[one_hot[e][evidence[e]]] = 1
        res[e] = r

    # infer values of all targets for given evidence set 
    infer = VariableElimination(model)
    for t in target:
        pred = infer.query([t], evidence=evidence, show_progress=False)
        res[t] = pred.values
        
    return res

In [27]:
def generate_sample_queries(model, test_samples, seed):
    
    np.random.seed(seed)
    
    all_states = model.states
    one_hot = {var:{state:i for i,state in enumerate(all_states[var])} for var in all_states}
    all_vars = list(all_states.keys())

    df = []

    for i, sample in test_samples.iterrows(): 

        # generate random mask
        mask = np.random.randint(0, 2, size = len(all_vars))
        while np.isclose(np.sum(mask), 0): # we will not be able to find zero-mask in ground-truth probability set 
            mask = np.random.randint(0, 2, size = len(all_vars))

        # create evidence and target set
        evidence = {}
        target = []
        for i in range(len(mask)):
            if mask[i] == 1:
                var_name = all_vars[i]
                evidence[var_name] = sample[var_name]
            else: 
                var_name = all_vars[i]
                target.append(var_name)
                
        res = pred_query(model, evidence, target)

        # add sample to dataset
        df.append(res)
        
    return df

In [29]:
sample_dataset = generate_sample_queries(GT_model, test_samples, 2022)
write_to_file(GT_model, sample_dataset, "data/asia_new/sample_test_set.txt")

## Independence relations

We can automatically extract all independence relations from the causal structure of the Asia model. 

In [35]:
ind = GT_model.get_independencies()

In [42]:
with open("data/asia_new/independencies.txt", 'w', encoding='utf-8') as file:
    file.write(str(ind))
    file.write("\n")

## Robustness experiments

We test how the performance of our models changes when the causal structure of the bayesian network is partially misspecified. To this end, we randomly select 5 edges to remove from, and 5 edges to add to the ground-truth structure, and we generate the independence relations from these new models.

### Remove edges

In [12]:
import numpy as np

In [17]:
edges = np.array(GT_model.edges)
np.random.seed(2022)
edge_idx = np.random.choice(np.arange(len(edges)), size=5, replace=False)
edge_rm = edges[edge_idx, :]
print(edge_rm)

[['tub' 'xray']
 ['lung' 'xray']
 ['smoke' 'lung']
 ['tub' 'dysp']
 ['asia' 'tub']]


The 5 edges above will be removed one by one from the ground-truth structure to obtain a partially incorrect causal structure. From this new structure, the independence relations can be automatically extracted as before, and can be used to train our neural networks.

In [27]:
def write_ind_file_rm(GT_model, i, u, v):
    model = GT_model.copy()
    with open("data/asia/remove_edges/ind_r"+str(i)+".txt", mode="w", encoding="utf-8") as file:
        model.remove_edge(u, v)
        ind = model.get_independencies()
        file.write(str(ind))
        file.write("\n")

In [28]:
write_ind_file_rm(GT_model, 1, "tub", "xray")
write_ind_file_rm(GT_model, 2, "lung", "xray")
write_ind_file_rm(GT_model, 3, "smoke", "lung")
write_ind_file_rm(GT_model, 4, "tub", "dysp")
write_ind_file_rm(GT_model, 5, "asia", "tub")

### Add edges

In [32]:
nodes = np.array(GT_model.nodes)
print(nodes)

['asia' 'tub' 'smoke' 'lung' 'bronc' 'xray' 'dysp']


We need to randomly select 5 pairs of nodes (= a random edge) from the list above, keeping the following conditions in mind: 
1. the edge is not present in the ground-truth DAG
2. the inverted edge is not present in the ground-truth DAG
3. adding the edge does not introduce any cycles in the DAG

We print the first 20 randomly chosen edges and manually select the first 5 that adhere to the criteria above.

In [33]:
np.random.seed(2022)
for i in range(20):
    node_names = np.random.choice(nodes, size = 2, replace=False)
    print(node_names)

['smoke' 'dysp']
['lung' 'xray']
['dysp' 'asia']
['tub' 'bronc']
['bronc' 'tub']
['lung' 'bronc']
['dysp' 'lung']
['dysp' 'asia']
['lung' 'xray']
['tub' 'bronc']
['lung' 'dysp']
['lung' 'smoke']
['asia' 'dysp']
['asia' 'xray']
['lung' 'asia']
['tub' 'smoke']
['bronc' 'xray']
['dysp' 'bronc']
['smoke' 'tub']
['asia' 'dysp']


The selected edges are the following: 
1. smoke -> dysp
2. tub -> bronc
3. lung -> bronc
4. asia -> dysp
5. asia -> xray

In [34]:
def write_ind_file_add(GT_model, i, u, v):
    model = GT_model.copy()
    with open("data/asia/add_edges/ind_a"+str(i)+".txt", mode="w", encoding="utf-8") as file:
        model.add_edge(u, v)
        ind = model.get_independencies()
        file.write(str(ind))
        file.write("\n")

In [35]:
write_ind_file_add(GT_model, 1, "smoke", "dysp")
write_ind_file_add(GT_model, 2, "tub", "bronc")
write_ind_file_add(GT_model, 3, "lung", "bronc")
write_ind_file_add(GT_model, 4, "asia", "dysp")
write_ind_file_add(GT_model, 5, "asia", "xray")